## 1. Character Image Generation
to ensure the consistency of characters across the trailers, we need the character portraits to be used as reference in the video generator. Here, we take description out of the generated movie trailer scripts and feed it to FLUX image generation model. 

In [4]:
from together import Together

In [5]:
api_key = ""  # Replace this with your actual API key

In [17]:
mara_description = 'a beautiful woman in her early 30s with long blond hair and blue eyes. Her face seems determined, intelligent, but also lonely. She looks like a marine biologist'
elias_description = 'an old man in his 60s as a lighthouse keeper with wrinkles and white hair. His face and skin are roughed up due to sunlight and work at sea'
theo_description = 'a man in his early 30s as a research assistant. He looks handsome, confident, friendly, and intelligent.'

In [18]:
prompt = f"Create a movie realistic full-body portrait image of {theo_description}"
add_on = ", photo realistic, perfect anatomy, perfect proportions, ultra-detailed, detailed realistic photo, incredibly lifelike, insanely detailed and intricate, extremely detailed, extremely high-resolution details, shallow depth of field, HDR, UHD, 32K"

after playing around with different models, the images created by the `FLUX.1-schnell-Free` are not realistic portrait at all and look rather cartoonish despite the prompt above specifying that it should be realistic. Thus, we tried out different models also. 

`FLUX.1 Kontext pro` model offers a very realistic look and is accurate to the prompt that is given. However, it is quite pricy at 4 cents per 1M pixel. That is approx. twice as expensive as `FLUX.1 dev`. The original model of the latter creates pictures that looks plastic-y, contains features that tells us that this is AI generated, similar to something out of MrBeast's YouTube thumbnail. However, when we add the weight adjustment LoRA from https://huggingface.co/XLabs-AI/flux-RealismLora, it looks pretty decent and is the best bang for the buck.

In [16]:
client = Together(api_key=api_key)
response = client.images.generate(
    prompt=prompt,
    model="black-forest-labs/FLUX.1-dev-lora",
    steps=33,
    n=1,
    response_format="url",
    image_loras=[
        {
            "path": "https://huggingface.co/XLabs-AI/flux-RealismLora",
            "scale": 1,
        }
    ],
)

KeyboardInterrupt: 

In [15]:
print(response.data[0].url)

https://api.together.ai/shrt/OtTuKSxNrahoHy8x


## 2. Shots Generation
After we have established the looks of our characters, and since we already have a very detailed script with description of scene composition, duration, etc., we will use the movie script to generate storyboard photos first, using our character portraits as a reference image. The reason is that the current text-to-video models doesn't support multiple image reference yet, and we need that to keep scene consistency. For this, we use the `FLUX 1 Kontext Pro` model since it is the most realistic. The generated storyboard pictures are then fed into the Image2Video model to generate the full 6-second shots, using the same prompt as the one given to generate the storyboard.

After testing out different models, such as Google Veo 2, Runway, Pika, Kling, and Hailuo 2, the best-performing model is the state-of-the-art Hailuo 2 with insanely accurate physics and very high-quality lighting. 

In [23]:
fantasy_vision_style = "Genre: Low Fantasy / Mystery Visual Language: Grounded naturalism meets reverent folklore Color Palette: Cold blue-greys, muted greens, fog whites, oil-lamp amber Lighting Style: Overcast daylight, candle/oil lamp interiors, minimal fill"

fantasy_scene_detail_1 = "EXT. CLIFFSIDE - DUSK — WIDE STATIC The lighthouse rises from the mist-slicked cliff like an ancient tooth — black iron and weatherworn stone against a slate-colored sky. Tufts of grass bend under steady wind. Ocean far below is barely visible, just sound and shimmer. Sound Design: A low ambient bed — wind over grass, gulls distant, a buoy bell tolls once. Music: A single bowed metal drone begins to swell from silence — low, dissonant, with the hiss of surf. Text Overlay (drifting in like mist): The sea keeps what it is owed."


prompt_text = f"Generate a shot for a fantasy film trailer with the vision style: {fantasy_vision_style}, and the following details: {fantasy_scene_detail_1}"
print(prompt_text)

Generate a shot for a fantasy film trailer with the vision style: Genre: Low Fantasy / Mystery Visual Language: Grounded naturalism meets reverent folklore Color Palette: Cold blue-greys, muted greens, fog whites, oil-lamp amber Lighting Style: Overcast daylight, candle/oil lamp interiors, minimal fill and the following details: EXT. CLIFFSIDE - DUSK — WIDE STATIC The lighthouse rises from the mist-slicked cliff like an ancient tooth — black iron and weatherworn stone against a slate-colored sky. Tufts of grass bend under steady wind. Ocean far below is barely visible, just sound and shimmer. Sound Design: A low ambient bed — wind over grass, gulls distant, a buoy bell tolls once. Music: A single bowed metal drone begins to swell from silence — low, dissonant, with the hiss of surf. Text Overlay (drifting in like mist): The sea keeps what it is owed.


In [25]:
from runwayml import RunwayML, TaskFailedError
from pathlib import Path
import mimetypes
import base64

def to_data_uri(path: str) -> str:
    """
    Read *path* (relative or absolute), base-64-encode its contents,
    and prefix with the correct MIME type, returning a ready-to-use
    data-URI string.
    """
    p = Path(path).expanduser().resolve()
    mime_type, _ = mimetypes.guess_type(p.name)
    if mime_type is None:
        # Fallback if mimetypes can’t figure it out
        mime_type = "application/octet-stream"
    data = base64.b64encode(p.read_bytes()).decode("ascii")
    return f"data:{mime_type};base64,{data}"

In [27]:
img_path = 'assets/storyboard/fantasy/lighthouse-opening.png'
data_uri = to_data_uri(img_path)

In [ ]:
runway_api_key = "YOUR_API_KEY"
client = RunwayML(api_key=runway_api_key)

try:
  task = client.image_to_video.create(
    model='gen4_turbo',
    # Point this at your own image file
    prompt_image=data_uri,
    prompt_text=prompt_text,
    ratio='1280:720',
    duration=5,
  ).wait_for_task_output()

  print('Task complete:', task)
except TaskFailedError as e:
  print('The video failed to generate.')
  print(e.task_details)

BadRequestError: Error code: 400 - {'error': 'You do not have enough credits to run this task.', 'docUrl': 'https://docs.dev.runwayml.com/api'}